In [ ]:
import mysql.connector
import requests

In [ ]:
conn_relacional = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password="0101",
    database="relacional",
)

conn_dimensional = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password="0101",
    database="dimensional",
)

cursor_relacional = conn_relacional.cursor()
cursor_dimensional = conn_dimensional.cursor()

In [ ]:
def pais_para_continente(pais):
    url = f"https://restcountries.com/v3.1/translation/{pais.lower()}"
    response = requests.get(url)

    if response.status_code != 200:
        return None
    
    data = response.json()[0]
    continente = data['region']

    traducao = {
        'Africa': 'África',
        'Americas': 'América',
        'Europe': 'Europa',
        'Asia': 'Ásia',
        'Oceania': 'Oceania',
        'Antarctic': 'Antártida'
    }
    return traducao.get(continente, continente)

## Localidade

In [ ]:
cursor_relacional.execute(
    "SELECT idPais, nmPais FROM pais"
)
paises = cursor_relacional.fetchall()

continentes = []
for id_pais, nome_pais in paises:
    continentes.append(pais_para_continente(nome_pais))

cursor_dimensional.executemany(
    "INSERT INTO d_localidade (idLocalidade, pais, continente) VALUES (%s, %s, %s)",
    [(pais[0], pais[1], continentes[0]) for pais in paises],
)

## Pagamento

In [ ]:
# campos: idPagamento, vlrRecebido
cursor_relacional.execute(
    "SELECT idPagamento, vlrRecebido FROM pagamento",
)
pagamentos = cursor_relacional.fetchall()

cursor_dimensional.executemany(
    "INSERT INTO d_Pagamento (idPagamento, vlrRecebido) VALUES (%s,%s)",
    pagamentos,
)

## Plano

In [ ]:
cursor_relacional.execute(
    "SELECT idPlano, nmPlano FROM plano",
)
planos = cursor_relacional.fetchall()

cursor_dimensional.executemany(
    "INSERT INTO d_plano (idPlano, nmPlano) VALUES (%s, %s)",
    planos
)

## Assinatura

In [ ]:
import collections
import datetime

data_lookup = {}
cursor_dimensional.execute("SELECT idData, ano, mes FROM d_data WHERE dia = 1")
for id_data, ano, mes in cursor_dimensional.fetchall():
    data_lookup[(ano, mes)] = id_data

localidade_lookup = {}
cursor_dimensional.execute("SELECT idLocalidade, pais, continente FROM d_localidade")
for id_localidade, pais, continente in cursor_dimensional.fetchall():
    localidade_lookup[(pais, continente)] = id_localidade

cursor_relacional.execute(
    """
    SELECT
        u.dtNascimento,
        p.nmPais,
        a.dtInicio
    FROM assinatura a
    JOIN usuario u ON u.idUsuario = a.Usuario_idUsuario
    JOIN pais p ON p.idPais = u.pais_idPais
    """
)
assinaturas = cursor_relacional.fetchall()

assinaturas_agregadas = collections.defaultdict(int)
data_atual = datetime.datetime.now()

for dt_nascimento, pais, dt_inicio in assinaturas:
    idade_usuario = data_atual.year - dt_nascimento.year - \
        ((data_atual.month, data_atual.day) < (dt_nascimento.month, dt_nascimento.day))
    faixa_etaria_usuario = idade_usuario // 10

    continente = pais_para_continente(pais)
    id_localidade = localidade_lookup.get((pais, continente))

    id_data = data_lookup.get((dt_inicio.year, dt_inicio.month))

    assinaturas_agregadas[(id_data, id_localidade, faixa_etaria_usuario)] += 1

cursor_dimensional.executemany(
    'INSERT INTO f_assinatura (idAssinatura, numAssinantes, faixa_etaria, d_Localidade_idLocalidade, d_Data_idData) VALUES (%s, %s, %s, %s, %s)',
    [
        (i, num_assinaturas, faixa_etaria, id_localidade, id_data)
        for i, ((id_data, id_localidade, faixa_etaria), num_assinaturas) in enumerate(assinaturas_agregadas.items(), start=1)
    ]
)

## Receita

In [ ]:
cursor_relacional.execute(
    """SELECT idPagamento, vlrRecebido, Assinatura_idAssinatura, dtPagamento FROM pagamento""",
)
pagamentos = cursor_relacional.fetchall()

dados_receita = []
for id_pagamento, valor_recebido, id_assinatura, dt_pagamento in pagamentos:
    # Assinatura
    cursor_relacional.execute(
        """SELECT Plano_idPlano FROM assinatura WHERE idAssinatura=%s""",
        (id_assinatura,)
    )
    plano_id = cursor_relacional.fetchone()[0]
    
    # idPlano
    cursor_dimensional.execute(
        """SELECT idPlano FROM d_plano WHERE idPlano=%s""",
        (plano_id,)
    )
    d_plano_id = cursor_dimensional.fetchone()[0]
    
    # idPagamento
    cursor_dimensional.execute(
        """SELECT idPagamento FROM d_pagamento WHERE idPagamento=%s""",
        (id_pagamento,)
    )
    d_pagamento_id = cursor_dimensional.fetchone()[0]
    
    # idData
    cursor_dimensional.execute(
        """
        SELECT idData FROM d_data
        WHERE mes=%s AND ano=%s
        """,
        (dt_pagamento.month, dt_pagamento.year)
    )
    d_data_id = cursor_dimensional.fetchall()
    print(dt_pagamento.month, dt_pagamento.year, len(d_data_id))
    
    dados_receita.append(
        (id_pagamento, valor_recebido, d_plano_id, d_pagamento_id, d_data_id)
    )

In [ ]:
conn_dimensional.commit()
